In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [215]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [216]:
tf.keras.backend.clear_session()

In [217]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
start_index = 60
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [218]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Unnamed: 0,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.334536,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,247.134
1,2003-2,0.756708,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.335670,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,305.708
2,2003-3,0.756879,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.336553,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,284.168
3,2003-4,0.757050,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.336531,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,286.594
4,2003-5,0.757222,0.648337,-5599.317941,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.336406,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,312.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.622619,1250.988075,NaN,NaN,NaN,NaN,NaN,633.858
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.621866,1247.030458,NaN,NaN,NaN,NaN,NaN,612.428
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.620866,1244.634022,NaN,NaN,NaN,NaN,NaN,579.831
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.619585,1241.214294,NaN,NaN,NaN,NaN,NaN,559.950


In [219]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.968698,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-1.575817,-2.202855,-0.752424,-1.806910,-1.971018,-3.111816,-2.777792
1,-1.943192,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-1.560482,-2.182122,-0.759292,-1.786501,-1.919572,-3.024730,-2.713279
2,-1.917685,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-1.548550,-2.161328,-0.766160,-1.766091,-1.868126,-2.937643,-2.648766
3,-1.892178,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-1.548848,-2.156390,-0.773028,-1.745682,-1.816680,-2.850557,-2.584252
4,-1.866672,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-1.550544,-2.136466,-0.779896,-1.725272,-1.765234,-2.763471,-2.519739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.081278,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.208028,1.070020,0.646276,0.920674,-0.608959,-0.023342,0.468241
200,1.064113,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.256499,1.058717,0.633557,0.902641,-0.600196,-0.045810,0.445578
201,1.046949,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.305916,1.081210,0.620837,0.884609,-0.591433,-0.068278,0.422914
202,1.029785,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,1.356297,1.085207,0.608118,0.866576,-0.582670,-0.090746,0.400250


In [220]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      305.708
1      284.168
2      286.594
3      312.455
4      299.651
        ...   
235    612.428
236    579.831
237    559.950
238    559.950
239        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 240, dtype: float64

In [221]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
60,-0.617077,-0.537241,-0.615167,0.042292,0.488894,-0.717137,-0.601292,-0.315948,0.240813,-0.757795,-0.238292,-1.028493,-0.822444,0.927636,0.275859,-0.396951
61,-0.598696,-0.479316,-0.525870,0.011943,-0.072763,-0.714911,-0.580436,-0.286095,0.246566,-0.727075,-0.209865,-1.025680,-0.813752,0.973169,0.285191,-0.394134
62,-0.580315,-0.466312,-0.510476,-0.019339,-0.316940,-0.694168,-0.559580,-0.256243,0.315327,-0.690455,-0.153086,-1.022868,-0.805061,1.018702,0.294523,-0.391316
63,-0.561934,-0.438778,-0.537853,-0.059264,0.180777,-0.700209,-0.538723,-0.226390,0.356189,-0.651003,-0.114447,-1.020055,-0.796370,1.064234,0.303855,-0.388499
64,-0.543553,-0.457839,-0.446299,-0.104671,-0.212484,-0.660717,-0.517867,-0.196537,0.524165,-0.607147,-0.062262,-1.017242,-0.787679,1.109767,0.313188,-0.385682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.334554,-0.742853,0.615006,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806,0.717612,-0.030629,1.239700,1.136971,-0.986304,0.021589,0.677638
164,1.337445,-0.787367,0.546355,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523,0.715300,-0.017187,1.195852,1.142243,-0.994112,0.041435,0.688796
165,1.340335,-0.757940,0.421319,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175,0.716462,-0.003286,1.152004,1.147515,-1.001919,0.061280,0.699954
166,1.343226,-0.917469,0.661467,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146,0.715461,0.000561,1.108156,1.152788,-1.009727,0.081125,0.711112


In [222]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

60     402.782000
61     429.874000
62     484.463000
63     498.487000
64     506.647000
          ...    
163    411.310457
164    406.131343
165    400.952229
166    395.773114
167    390.594000
Name: Minas Gerais - Consumo de Cimento (t), Length: 108, dtype: float64

In [223]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [254]:
reshaped_train, reshaped_target = create_batches(train_input, train_target, 36, 60)
reshaped_train.shape

(72, 36, 16)

In [255]:
# input para treinamento
test_input = input_data.iloc[train_split-36:split_index + 1]
test_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
132,0.558603,1.590448,0.225519,-0.298056,-0.311347,-0.077688,0.623805,0.982447,0.041401,1.013808,0.681029,1.243600,0.846926,0.022034,0.277116,0.755622
133,0.562216,1.702213,0.413533,-0.281023,-0.313831,-0.030826,0.635788,0.979099,-0.019620,1.008961,0.654027,1.276498,0.858253,-0.030478,0.243655,0.739933
134,0.565829,1.677438,0.431751,-0.265304,0.006739,-0.076111,0.647771,0.975750,0.029284,1.009201,0.625159,1.309396,0.869581,-0.082991,0.210193,0.724244
135,0.569442,1.638599,0.414613,-0.251648,-0.486922,-0.205002,0.659754,0.972402,0.072516,1.006419,0.595518,1.342293,0.880908,-0.135503,0.176731,0.708555
136,0.573055,1.691843,0.566348,-0.231997,-0.559925,-0.132229,0.671737,0.969054,-0.064532,1.003323,0.565555,1.375191,0.892235,-0.188016,0.143269,0.692865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.081278,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.208028,1.070020,0.646276,0.920674,-0.608959,-0.023342,0.468241
200,1.064113,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.256499,1.058717,0.633557,0.902641,-0.600196,-0.045810,0.445578
201,1.046949,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.305916,1.081210,0.620837,0.884609,-0.591433,-0.068278,0.422914
202,1.029785,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,1.356297,1.085207,0.608118,0.866576,-0.582670,-0.090746,0.400250


In [256]:
reshaped_test, reshaped_test_target = create_batches(test_input, target_data, 36, 132)
reshaped_test.shape 

(36, 36, 16)

In [258]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * 6 + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [298]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 10)
    target,target_val = validation_splitter(train_target, 10)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36, activation='tanh'),
#         tf.keras.layers.Dropout(0.3),
#         tf.keras.layers.Dense(74, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [299]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(5)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [ ]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3654598171, 350372021, 3558607853, 1623443814, 4262395417]


Step: 0 ___________________________________________


In [ ]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

In [ ]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [ ]:
errors, mae, mape = mae_mape_calculator(trained_model, 
                                        reshaped_test, 
                                        reshaped_test_target, 0)

In [ ]:
display(mae)
display(mape)

In [ ]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
            target_sum = 0
            prediction_sum = 0
            
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [ ]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 0)
display(mae)
display(mape)